# CMSC320 Final Tutorial
## David Martin

In [10]:
import requests
import pandas as pd

In [11]:
# Pull table for county calls for Henrico County
call_table = pd.read_csv("https://raw.githubusercontent.com/martindavid1995/Datasets/master/Open_View_-_Henrico_County_Calls_for_Service.csv")

